In [1]:
import os, sys, time, re, json
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import scipy.stats as ss
from functools import reduce
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib import style
style.use('ggplot')
style.use('seaborn-white')

/home/xiaojiashun/anaconda3/envs/SpatialScope/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
celltype_map = {'Astrocytes': 'Astrocytes',
                'Endothelial.cells': 'Endothelial',
                'Excitatory.neurons': 'Excitatory_neurons',
                'Inhibitory.neurons': 'Inhibitory_neurons',
                'Microglia': 'Microglia',
                'Oligodendrocytes': 'Oligodendrocytes',
                'OPCs...COPs': 'OPCs',
                'Pericytes': 'Pericytes'}

# Data preparation

## Merge 1M with 1000G reference panel

In [ ]:
### merge maf001 with brain by cell type
## 1000G maf001 snps
bims_df = pd.read_csv('../data/S-LDXR/1000G_EAS_EUR_maf0.01/all_chroms.bim', sep='\t')
maf001_allsnps = pd.read_csv('../data/S-LDXR/1000G_EAS_EUR_maf0.01/print_snps.txt', sep='\t', header=None)[0].values.tolist()
print(f'{len(maf001_allsnps)} SNPs in 1000G maf001')
for filename, celltype in celltype_map.items():
    t0 = time.time()
    for chrom in range(1, 23):
        savefile = f'../data/brain_data/gene_snp_pairs/{celltype}_pairs_chr{chrom}.csv'
        if os.path.exists(savefile):
            print(f'{celltype}_pairs_chr{chrom} saved.')
            break
        ## brain eqtl
        df_1m = pd.read_csv(f'../data/brain_eqtl/{filename}.{chrom}.gz', sep=' ', compression='gzip', header=None)
        df_1m.columns = ['GENEID_NAME', 'SNP', 'DIST', 'P', 'BETA']
        df_1m['GENEID'] = df_1m['GENEID_NAME'].apply(lambda x: x.split('_')[0])
        df_1m['GENENAME'] = df_1m['GENEID_NAME'].apply(lambda x: x.split('_')[1])
        print(f'Loaded brain eqtl [{celltype} chrom {chrom}] {df_1m.shape}')
        ## merge brain eqtl with 1000G maf001
        df_1m_in_maf001_snps = pd.merge(bims_df.loc[bims_df.CHR==chrom], df_1m, on='SNP') # based on 1kg
        print(f'{len(np.unique(df_1m.SNP.values))} SNPs in original df_1m, left {len(np.unique(df_1m_in_maf001_snps.SNP.values))} SNPs after merging with 1000G maf001')
        ## save basic info columns
        df_1m_in_maf001_snps[['SNP', 'CHR', 'BP', 'CM', 'A1', 'A2', 'GENEID', 'GENENAME', 'DIST']].to_csv(savefile, sep='\t', index=None)
        print(f'Saved df_1m_in_maf001_snps {df_1m_in_maf001_snps.shape}')
    t1 = time.time()
    print(f'{celltype} | Time: {t1 - t0:.1f}s\n')

In [ ]:
### merge union of all pairs
t0 = time.time()
for chrom in range(1, 23):
    df1 = pd.read_csv(f'../data/brain_data/gene_snp_pairs/Astrocytes_pairs_chr{chrom}.csv', sep='\t')
    for celltype in ['Endothelial', 'Excitatory_neurons', 'Inhibitory_neurons', 'Microglia', 'Oligodendrocytes', 'OPCs', 'Pericytes']:
        df2 = pd.read_csv(f'../data/brain_data/gene_snp_pairs/{celltype}_pairs_chr{chrom}.csv', sep='\t')
        print(celltype, df1.shape, df2.shape)
        df1 = pd.merge(df1, df2, how='outer')
    print(f'chr {chrom}: {df1.shape}')
    df1 = df1.sort_values(by=['CHR', 'BP'])
    df1.to_csv(f'../data/brain_data/gene_snp_pairs/all_celltype_pairs_chr{chrom}.csv', sep='\t', index=False)
    t1 = time.time()
    print(f'Time: {t1 - t0:.1f}s')

## Create annotation matrix for calculating LD score

In [ ]:
### create annotation matrix from pairs
print_snps_all_chrom = []
t0 = time.time()
for chrom in range(1, 23):
    print(f'*** CHR {chrom} ***')
    pairs_1m = pd.read_csv(f'../data/brain_data/gene_snp_pairs/all_celltype_pairs_chr{chrom}.csv', sep='\t')
    print(pairs_1m.shape)
    gene_ids_1m = np.unique(pairs_1m.GENEID.values)
    print(f'{len(gene_ids_1m)} gene_ids in 1M all celltype pairs')
    savefile = f'../data/brain_data/annotations/{chrom}.annot.gz'
    gene_snp_map_1m = pairs_1m[pairs_1m.CHR==chrom][['SNP', 'GENEID']].groupby('GENEID')['SNP'].apply(list).to_dict()
    annot = pairs_1m.loc[pairs_1m.CHR==chrom][['CHR', 'BP', 'SNP', 'CM']]
    annot = annot.drop_duplicates(subset=['SNP'])
    annot['base'] = 1
    for gene, snps in tqdm(gene_snp_map_1m.items()):
        annot[gene] = 0
        annot.loc[annot.SNP.isin(snps), gene] = 1
    genes_per_snp = annot.iloc[:,5:].sum(1)
    snps_per_gene = annot.iloc[:,5:].sum(0)
    print(f'{min(genes_per_snp)}-{max(genes_per_snp)} genes per snp, {min(snps_per_gene)}-{max(snps_per_gene)} snps per gene')
    annot.to_csv(savefile, sep='\t', index=False, compression='gzip')
    ## save print snps
    print_snps = pd.unique(pairs_1m.SNP.values).tolist()
    print(f'{len(print_snps)} snps in pairs_1m.\n')
    with open(f'../data/brain_data/annotations/{chrom}.print_snps.txt', 'w') as f:
        for x in print_snps:
            f.write(x + '\n')
    print_snps_all_chrom.extend(print_snps)
    t1 = time.time()
    print(f'Time: {t1 - t0:.1f}s')
with open('../data/brain_data/annotations/all_chroms.print_snps.txt', 'w') as f:
    for x in print_snps_all_chrom:
        f.write(x + '\n')

## Process GTEx data

In [ ]:
## original gtex data
t0 = time.time()
df_gtex = pd.read_csv('../data/GTEx/brain_cortex/GTEx_Analysis_v8_QTLs_GTEx_Analysis_v8_eQTL_all_associations_Brain_Cortex.allpairs.txt.gz', sep='\t', compression='gzip')
t1 = time.time()
print(f'Time loading gtex: {t1 - t0:.1f}s')
print(df_gtex.shape, np.unique(df_gtex.variant_id.values).shape)

In [32]:
## gtex variants lookup
t0 = time.time()
lookup = pd.read_csv('../data/GTEx/references_v8_GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.lookup_table.txt.gz', sep='\t', compression='gzip')
t1 = time.time()
print(f'Time loading gtex lookup table {lookup.shape}: {t1 - t0:.1f}s')

Time loading gtex lookup table (46569704, 8): 180.4s


In [34]:
## merge
df_gtex_with_rsid = pd.merge(df_gtex, lookup[['variant_id', 'variant_pos', 'rs_id_dbSNP151_GRCh38p7']], on='variant_id')
print(f'df_gtex_with_rsid.shape = {df_gtex_with_rsid.shape}')
print(np.unique(df_gtex_with_rsid.rs_id_dbSNP151_GRCh38p7.values).shape)
df_gtex_with_rsid.to_csv('../data/GTEx/brain_cortex/all_associations.txt.gz', sep='\t', index=False, compression='gzip')

df_gtex_with_rsid.shape = (182641182, 11)
(10434926,)


In [ ]:
## gtex genes
gtf = pd.read_csv('../data/GTEx/references_v8_gencode.v26.GRCh38.genes.gtf', sep='\t', skiprows=[0,1,2,3,4,5], header=None)
print(gtf.shape)
gtf['gene_id'] = gtf[8].map(lambda x: x.split(';')[0].replace('gene_id ', '').replace('"', '').strip())
gtf['gene_name'] = gtf[8].map(lambda x: x.split(';')[3].replace('gene_name ', '').replace('"', '').strip())

In [3]:
### process gtex
df_gtex_with_rsid = pd.read_csv('../data/GTEx/brain_cortex/all_associations.txt.gz', sep='\t')
df_gtex_with_rsid['ensemble_id'] = df_gtex_with_rsid['gene_id'].map(lambda x: x.split('.')[0])
df_gtex_with_rsid['A2'] = df_gtex_with_rsid.variant_id.map(lambda x: x.split('_')[3])
df_gtex_with_rsid['A1'] = df_gtex_with_rsid.variant_id.map(lambda x: x.split('_')[2])
df_gtex = df_gtex_with_rsid[['rs_id_dbSNP151_GRCh38p7', 'ensemble_id', 'A1', 'A2', 'slope', 'slope_se', 'pval_nominal']]
df_gtex.columns = ['SNP_GTEX', 'GENEID_GTEX', 'A1_GTEX', 'A2_GTEX', 'BETA_GTEX', 'SE_GTEX', 'PVAL_GTEX']

## Merge sc-brain and GTEx data by cell type

In [4]:
snp_pos = pd.read_csv(f'../data/brain_eqtl/snp_pos.txt.gz', sep='\t', compression='gzip')
print(snp_pos.shape)

(5352389, 6)


In [ ]:
for filename, celltype in celltype_map.items():
    print(f'***** {celltype} *****')
    df_1m_allchrom, pairs_allchrom = [], []
    t0 = time.time()
    for chrom in range(1, 23):
        print(f'--- CHR {chrom} ---')
        df_1m = pd.read_csv(f'../data/brain_eqtl/{filename}.{chrom}.gz', sep=' ', compression='gzip', header=None)
        df_1m.columns = ['GENEID_NAME', 'SNP', 'DIST', 'PVAL', 'BETA']
        df_1m['GENEID'] = df_1m['GENEID_NAME'].apply(lambda x: x.split('_')[1])
        df_1m['GENENAME'] = df_1m['GENEID_NAME'].apply(lambda x: x.split('_')[0])
        print(f'Loaded brain sc-eqtl {celltype} chr {chrom} {df_1m.shape}')
        df_1m = pd.merge(df_1m, snp_pos[['SNP', 'effect_allele', 'other_allele']], on='SNP', how='inner')
        df_1m['SE'] = abs(df_1m.BETA/ss.norm.ppf(df_1m.PVAL/2))
        df_1m = df_1m[['SNP', 'DIST', 'GENEID', 'GENENAME', 'other_allele', 'effect_allele', 'BETA', 'SE', 'PVAL']]
        df_1m.columns = ['SNP_SC', 'DIST_SC', 'GENEID_SC', 'GENENAME_SC', 'A1_SC', 'A2_SC', 'BETA_SC', 'SE_SC', 'PVAL_SC']
        df_1m_allchrom.append(df_1m)
        ## load pairs (1M merged with 1000G ref)
        pairs = pd.read_csv(f'../data/brain_data/gene_snp_pairs/{celltype}_pairs_chr{chrom}.csv', sep='\t')
        pairs.columns = ['SNP_SC', 'CHR_1KG', 'BP_1KG', 'CM_1KG', 'A1_1KG', 'A2_1KG', 'GENENAME_SC', 'GENEID_SC', 'DIST_SC']
        print(f'Brain sc-eqtl pairs (merged with 1000G maf0.01 ref): {pairs.shape}')
        pairs_allchrom.append(pairs)
    df_1m = pd.concat(df_1m_allchrom)
    pairs = pd.concat(pairs_allchrom)
    t1 = time.time()
    print(f'All chrom: df_1m {df_1m.shape}, pairs {pairs.shape} [{t1 - t0:.1f}s]')

    ## merge with gtex
    t0 = time.time()
    merged_sc_gtex = pd.merge(df_1m, df_gtex, left_on=['SNP_SC', 'GENEID_SC'], right_on=['SNP_GTEX', 'GENEID_GTEX'], how='inner')
    print(f'Merged SC and GTEX: merged_sc_gtex {merged_sc_gtex.shape}')
    ## merge with all_pairs (1m merged with 1000G maf0.01)
    merged_sc_gtex_maf001 = pd.merge(pairs[['SNP_SC', 'A1_1KG', 'A2_1KG', 'GENEID_SC']], merged_sc_gtex, on=['SNP_SC', 'GENEID_SC'], how='inner')
    t1 = time.time()
    print(f'Merged SC, GTEX and maf001: merged_sc_gtex {merged_sc_gtex_maf001.shape} [{t1 - t0:.1f}s]')

    ## remove ambiguous snps (A1A2 != 1000G A1A2)
    tmp = merged_sc_gtex_maf001.reset_index(drop=True)
    l0 = len(tmp)
    snps_rm_ids1 = ~(((tmp.A1_SC==tmp.A1_1KG) & (tmp.A2_SC==tmp.A2_1KG)) | ((tmp.A1_SC==tmp.A2_1KG) & (tmp.A2_SC==tmp.A1_1KG)))
    snps_rm_ids3 = ~(((tmp.A1_GTEX==tmp.A1_1KG) & (tmp.A2_GTEX==tmp.A2_1KG)) | ((tmp.A1_GTEX==tmp.A2_1KG) & (tmp.A2_GTEX==tmp.A1_1KG)))
    snps_rm_ids = (snps_rm_ids1 | snps_rm_ids3)
    tmp = tmp.loc[~snps_rm_ids]
    print(f'Removed {l0 - len(tmp)} ambiguous SNPs.')
    ## reverse A1/A2 based on 1000G A1/A2
    rev_ids1 = (tmp.A2_SC!=tmp.A2_1KG)
    rev_ids3 = (tmp.A2_GTEX!=tmp.A2_1KG)
    tmp.loc[rev_ids1, ['A1_SC', 'A2_SC']] = tmp.loc[rev_ids1, ['A2_SC', 'A1_SC']].values
    tmp.loc[rev_ids3, ['A1_GTEX', 'A2_GTEX']] = tmp.loc[rev_ids3, ['A2_GTEX', 'A1_GTEX']].values
    tmp.loc[rev_ids1, 'BETA_SC'] = -tmp.loc[rev_ids1, 'BETA_SC'].values
    tmp.loc[rev_ids3, 'BETA_GTEX'] = -tmp.loc[rev_ids3, 'BETA_GTEX'].values
    print(f'Reversed A1/A2: SC {sum(rev_ids1)}, GTEX {sum(rev_ids3)}')
    tmp = tmp.drop(['A1_SC', 'A2_SC', 'A1_GTEX', 'A2_GTEX', 'SNP_GTEX', 'GENEID_GTEX'], axis=1)
    tmp.to_csv(f'../data/brain_data/merged_sumstats/{celltype}.txt.gz', sep='\t', compression='gzip')
    print(f'Saved combined sumstats ({tmp.shape}).')

    ## significance overlap
    pval_thres = 1e-5
    sig_1m = tmp.loc[tmp.PVAL_SC < pval_thres]
    sig_gtex = tmp.loc[tmp.PVAL_GTEX < pval_thres]
    # sig genes
    sig_1m_genes = np.unique(sig_1m.GENEID_SC.values)
    sig_gtex_genes = np.unique(sig_gtex.GENEID_SC.values)
    sig_common_genes = np.intersect1d(sig_1m_genes, sig_gtex_genes)
    sig_only_1m_genes = np.setdiff1d(sig_1m_genes, sig_gtex_genes)
    sig_only_gtex_genes = np.setdiff1d(sig_gtex_genes, sig_1m_genes)
    print(f'1M vs gtex | Common: {len(sig_common_genes)}, Only 1M: {len(sig_only_1m_genes)}, Only gtex: {len(sig_only_gtex_genes)}')
    ## effect size direction concordance
    tmp2 = tmp.loc[(tmp.PVAL_SC < pval_thres) & (tmp.PVAL_GTEX < pval_thres)]
    same_direction = sum((tmp2.BETA_SC > 0) & (tmp2.BETA_GTEX > 0)) + sum((tmp2.BETA_SC < 0) & (tmp2.BETA_GTEX < 0))
    print(f'Effect size (pval<1e-5): {same_direction}/{len(tmp2)}={same_direction/len(tmp2):.2f} same direction. [{t1 - t0:.1f}s]\n')

In [8]:
numsnps_per_gene_lists = {}
for filename, celltype in celltype_map.items():
    print(f'***** Process {celltype} *****')
    t0 = time.time()
    df = pd.read_csv(f'../data/brain_data/merged_sumstats/{celltype}.txt.gz', sep='\t', index_col=0)
    t1 = time.time()
    print(f'Loaded {celltype}: {t1 - t0:.1f}s')
    snps_per_gene = df[['SNP_SC', 'GENEID_SC']].groupby('GENEID_SC')['SNP_SC'].apply(list).to_dict()
    numsnps_per_gene = {k: len(v) for k, v in snps_per_gene.items()}
    numsnps_per_gene_list = list(numsnps_per_gene.values())
    numsnps_per_gene_lists[celltype] = numsnps_per_gene_list
    print(f'{celltype} | 5th-percentail {np.percentile(numsnps_per_gene_list, 5):.1f}, mean {np.mean(numsnps_per_gene_list):.1f}, median {np.median(numsnps_per_gene_list):.1f}\n')

***** Process Astrocytes *****
Loaded Astrocytes: 82.4s
Astrocytes | 5th-percentail 1905.3, mean 3600.4, median 3593.0

***** Process Endothelial *****
Loaded Endothelial: 86.7s
Endothelial | 5th-percentail 1891.3, mean 3600.1, median 3591.5

***** Process Excitatory_neurons *****
Loaded Excitatory_neurons: 110.6s
Excitatory_neurons | 5th-percentail 1899.0, mean 3590.8, median 3597.0

***** Process Inhibitory_neurons *****
Loaded Inhibitory_neurons: 97.5s
Inhibitory_neurons | 5th-percentail 1906.0, mean 3592.5, median 3606.0

***** Process Microglia *****
Loaded Microglia: 65.3s
Microglia | 5th-percentail 1874.9, mean 3595.2, median 3596.0

***** Process Oligodendrocytes *****
Loaded Oligodendrocytes: 78.7s
Oligodendrocytes | 5th-percentail 1888.0, mean 3570.2, median 3564.0

***** Process OPCs *****
Loaded OPCs: 78.2s
OPCs | 5th-percentail 1915.0, mean 3607.6, median 3606.0

***** Process Pericytes *****
Loaded Pericytes: 49.1s
Pericytes | 5th-percentail 1882.0, mean 3623.5, median 36

## Organize data by chrom and gene

In [2]:
bims_df = pd.read_csv('../data/S-LDXR/1000G_EAS_EUR_maf0.01/all_chroms.bim', sep='\t')
print(f'{len(bims_df)} SNPs in 1000G maf001.')

8534993 SNPs in 1000G maf001.


In [ ]:
### save sumstats by chrom and gene
numsnp_thres = 50
for chrom in range(22, 0, -1):
    savedir = f'../data/brain_data/merged_sumstats/by_chrom_gene/chr{chrom}'
    os.makedirs(savedir, exist_ok=True)
    for filename, celltype in celltype_map.items():
        t0 = time.time()
        print(f'***** Processing CHR {chrom} {celltype} *****')
        sumstat = pd.read_csv(f'../data/brain_data/merged_sumstats/{celltype}.txt.gz', sep='\t', compression='gzip')
        l0 = len(sumstat)
        sumstat_ = pd.merge(sumstat, bims_df[['SNP', 'CHR', 'BP']], left_on='SNP_SC', right_on='SNP')
        l1 = len(sumstat_)
        sumstat_ = sumstat_.loc[sumstat_.CHR==chrom]
        l2 = len(sumstat_)
        sumstat_ = sumstat_.drop_duplicates(subset=['SNP_SC', 'GENEID_SC'])
        l3 = len(sumstat_)
        snps_per_gene = sumstat_.groupby('GENEID_SC')['SNP_SC'].apply(list).to_dict()
        numsnps_per_gene = {k: len(v) for k, v in snps_per_gene.items()}
        numsnps_per_gene_df = pd.DataFrame({'GENEID': list(numsnps_per_gene.keys()), 'NUM_SNPS': list(numsnps_per_gene.values())})
        keep_gene_df = numsnps_per_gene_df.loc[numsnps_per_gene_df.NUM_SNPS>numsnp_thres]
        keep_sumstat = sumstat_.loc[sumstat_['GENEID_SC'].isin(keep_gene_df.GENEID)]
        l4 = len(keep_sumstat)
        print(f'original {l0} -> merge with maf001 {l1} -> chr{chrom} {l2} -> removed duplicates {l3} -> keep genes with at least snps {l4}')
        g = keep_sumstat.groupby(['GENEID_SC'])
        for i in tqdm(range(len(g.groups))):
            geneid = list(g.groups)[i]
            df = g.get_group(geneid)
            sumstat_1m = df[['CHR', 'BP', 'A1_1KG', 'A2_1KG', 'SNP_SC', 'GENEID_SC', 'GENENAME_SC', 'BETA_SC', 'SE_SC', 'PVAL_SC']]
            sumstat_gtex = df[['CHR', 'BP', 'A1_1KG', 'A2_1KG', 'SNP_SC', 'GENEID_SC', 'GENENAME_SC', 'BETA_GTEX', 'SE_GTEX', 'PVAL_GTEX']]
            sumstat_1m.columns = ['CHR', 'BP', 'A1', 'A2', 'SNP', 'GENEID', 'GENENAME', 'BETA', 'SE', 'PVAL']
            sumstat_gtex.columns = ['CHR', 'BP', 'A1', 'A2', 'SNP', 'GENEID', 'GENENAME', 'BETA', 'SE', 'PVAL']
            sumstat_1m.to_csv(f'{savedir}/{geneid}_{celltype}.csv', sep='\t', index=False)
            sumstat_gtex.to_csv(f'{savedir}/{geneid}_gtex.csv', sep='\t', index=False)
        t1 = time.time()
        print(f'Processed CHR {chrom} {celltype} {t1 - t0:.1f}s.\n')

## Calculate LD scores

In [ ]:
## command for running S-LDXR
for chrom in range(1, 23):
    command = f'../software/s-ldxr-0.3-beta/s-ldxr.py \
                --bfile ../data/brain_data/genotype/1000G.EUR.QC.{chrom} ../data/brain_data/genotype/1000G.EUR.QC.{chrom} \
                --print-snps ../data/brain_data/annotations/all_chroms.print_snps.txt \
                --annot ../data/brain_data/annotations/{chrom}.annot.gz \
                --ld-wind-cm 1.0 \
                --score standardized \
                --out ../data/brain_data/ldscore/EUR_EUR_std_chr{chrom}'
    print(command)

In [ ]:
for chrom in range(1, 23):
    gene_id_name = pd.read_csv(f'../data/brain/gene_id_name.{chrom}.csv', sep='\t')
    t0 = time.time()
    ld_df = pd.read_csv(f'../data/brain_data/ldscore/EUR_EUR_std_chr{chrom}_pop1.gz', sep='\t', compression='gzip')
    print(f'CHR {chrom} ld_df.shape: {ld_df.shape}')
    ld_genes = ld_df.columns[4:]
    print(f'{len(ld_genes)} genes in ld_df')
    files = glob.glob(f'../data/brain_data/merged_sumstats/by_chrom_gene/chr{chrom}/*_gtex.csv')
    for file in tqdm(files):
        geneid = file.split('/')[-1].split('_')[0]
        genename = gene_id_name.loc[gene_id_name.GENEID==geneid, 'GENENAME'].values[0]
        gtex_df = pd.read_csv(file, sep='\t')
        merged = pd.merge(gtex_df, ld_df[['SNP', genename]], on='SNP', how='inner')
        merged = merged.rename(columns={genename: 'LD'})
        merged.to_csv(f'../data/brain_data/merged_sumstats/by_chrom_gene/chr{chrom}/{geneid}_gtex_withld.csv', sep='\t', index=False)
    t1 = time.time()
    print(f'CHR {chrom} time: {t1 - t0:.1f}s')

## Average cell type proportions

In [5]:
avg_props = pd.read_csv('../data/cibersortx/cibersortx_8meanprops.csv', sep='\t')
avg_props

,cell_type,prop
0,Inhibitory neurons,0.102131
1,Oligodendrocytes,0.169932
2,Excitatory neurons,0.308489
3,Astrocytes,0.290534
4,OPCs,0.003799
5,Microglia,0.049073
6,Pericytes,0.024232
7,Endothelial,0.051812
